In [5]:
# ============================================================
# FLIGHT DISRUPTION INFERENCE (4-STAGE + META CLASSIFIER)
# USING CSV-BASED ENCODINGS
# MEMORY SAFE – LOAD & FREE MODELS ONE BY ONE
# ============================================================

import pandas as pd
import numpy as np
import joblib
import gc

# ============================================================
# 1. LOAD FEATURE LIST
# ============================================================

FEATURES  = [
    'DayOfWeek',
    'DayofMonth',
    'Month',
    'Distance',
    'CRSDepMin',
    'CRSArrMin',
    'ScheduledElapsedTime',
    'OriginReliability',
    'DestReliability',
    'CarrierReliability',
    'DepTimeOfDay_enc',
    'ArrTimeOfDay_enc'
]



# ============================================================
# 2. LOAD ENCODINGS FROM CSV
# ============================================================

carrier_df = pd.read_csv("C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\encodings\\carrier_reliability_encoding.csv")
origin_df  = pd.read_csv("C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\encodings\\origin_reliability_encoding.csv")
dest_df    = pd.read_csv("C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\encodings\\dest_reliability_encoding.csv")

carrier_map = dict(zip(carrier_df["UniqueCarrier"], carrier_df["CarrierReliability"]))
origin_map  = dict(zip(origin_df["Origin"], origin_df["OriginReliability"]))
dest_map    = dict(zip(dest_df["Dest"], dest_df["DestReliability"]))

# ============================================================
# 3. TIME OF DAY ENCODING (SAME AS TRAINING)
# ============================================================

def time_of_day(hour):
    if 5 <= hour < 11:
        return 0   # Morning
    elif 11 <= hour < 17:
        return 1   # Afternoon
    elif 17 <= hour < 22:
        return 2   # Evening
    else:
        return 3   # Night

# ============================================================
# 4. USER INPUT
# ============================================================

print("\n✈️ ENTER FLIGHT DETAILS")

carrier = input("Carrier (AA, DL, UA): ").upper().strip()
origin  = input("Origin Airport (JFK): ").upper().strip()
dest    = input("Destination Airport (LAX): ").upper().strip()

dep_hour = int(input("Departure hour (0–23): "))
arr_hour = int(input("Arrival hour (0–23): "))

# ============================================================
# 5. VALIDATION (CRITICAL)
# ============================================================

if carrier not in carrier_map:
    raise ValueError(f"Unknown carrier: {carrier}")

if origin not in origin_map:
    raise ValueError(f"Unknown origin airport: {origin}")

if dest not in dest_map:
    raise ValueError(f"Unknown destination airport: {dest}")

# ============================================================
# 6. PREPROCESS INPUT
# ============================================================

row = {
    "CarrierReliability": carrier_map[carrier],
    "OriginReliability": origin_map[origin],
    "DestReliability": dest_map[dest],
    "DepTimeOfDay_enc": time_of_day(dep_hour),
    "ArrTimeOfDay_enc": time_of_day(arr_hour),
}

# ensure exact feature alignment
for f in FEATURES:
    row.setdefault(f, 0)

X = pd.DataFrame([row])[FEATURES]

# ============================================================
# 7. STAGE-1: BASE MODELS (LOAD → PREDICT → FREE)
# ============================================================

meta_features = {}

# --- Diverted vs Others ---
clf_div = joblib.load("C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\class_4stage_models\\clf_diverted.pkl")
meta_features["p_div"] = clf_div.predict_proba(X)[0, 1]
del clf_div
gc.collect()

# --- On Time vs Delayed ---
clf_ot_del = joblib.load("C:\\Users\\HP\\OneDrive\\AppData\Desktop\\!!74 pr\\models\\class_4stage_models\\clf_ot_delayed.pkl")
meta_features["p_ot_del"] = clf_ot_del.predict_proba(X)[0, 1]
del clf_ot_del
gc.collect()

# --- Delayed vs Cancelled ---
clf_del_can = joblib.load("C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\class_4stage_models\\clf_del_cancelled.pkl")
meta_features["p_del_can"] = clf_del_can.predict_proba(X)[0, 1]
del clf_del_can
gc.collect()

# --- On Time vs Cancelled ---
clf_ot_can = joblib.load("C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\class_4stage_models\\clf_ot_cancelled.pkl")
meta_features["p_ot_can"] = clf_ot_can.predict_proba(X)[0, 1]
del clf_ot_can
gc.collect()

# ============================================================
# 7.5 SEGREGATION PROBABILITY STEP (MATCH META TRAINING)
# ============================================================

P_Diverted  = meta_features["p_div"]
P_Cancelled = (meta_features["p_del_can"] + meta_features["p_ot_can"]) / 2
P_Delayed   = (1 - meta_features["p_ot_del"]) * (1 - meta_features["p_del_can"])
P_OnTime    = meta_features["p_ot_del"] * meta_features["p_ot_can"]

X_meta = pd.DataFrame([{
    "P_Diverted":  P_Diverted,
    "P_Cancelled": P_Cancelled,
    "P_Delayed":   P_Delayed,
    "P_OnTime":    P_OnTime
}])

# ============================================================
# 8. META CLASSIFIER
# ============================================================



meta_clf = joblib.load("C:\\Users\\HP\\OneDrive\\AppData\\Desktop\\!!74 pr\\models\\meta_classifier\\meta_extratrees.pkl")

probs = meta_clf.predict_proba(X_meta)[0]
pred_idx = np.argmax(probs)

classes = ["Cancelled", "Delayed", "Diverted", "On Time"]
final_prediction = classes[pred_idx]

del meta_clf
gc.collect()

# ============================================================
# 9. OUTPUT
# ============================================================

print("\n🧠 FINAL DISRUPTION PREDICTION")
print("----------------------------------")
print(f"Carrier     : {carrier}")
print(f"Route       : {origin} → {dest}")
print(f"Departure   : {dep_hour}:00")
print(f"Arrival     : {arr_hour}:00")

print("\nClass Probabilities:")
for cls, p in zip(classes, probs):
    print(f"{cls:10s}: {p:.3f}")

print(f"\n📌 FINAL PREDICTION: **{final_prediction}**")


✈️ ENTER FLIGHT DETAILS



🧠 FINAL DISRUPTION PREDICTION
----------------------------------
Carrier     : AA
Route       : JFK → LAX
Departure   : 13:00
Arrival     : 15:00

Class Probabilities:
Cancelled : 0.275
Delayed   : 0.475
Diverted  : 0.012
On Time   : 0.238

📌 FINAL PREDICTION: **Delayed**
